In [1]:
import urllib.request
import requests
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import re
import time
from tqdm import tqdm
import pandas as pd
from collections import OrderedDict
import pymysql

In [2]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

# Weibo Spider

In [26]:
def createtableBloggers(tablename):
    
    try:
        conn = pymysql.connect('localhost', 'msba', 'hku2019', 'makeup')
        conn.ping(False)
        cur = conn.cursor()
        
        sqldrop = 'DROP TABLE IF EXISTS ' + tablename
        sqlcreate = 'CREATE TABLE ' + tablename + ' (id INT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, name VARCHAR(30) DEFAULT NULL, uid VARCHAR(10) DEFAULT NULL, gender VARCHAR(10) DEFAULT NULL, region VARCHAR(5) DEFAULT NULL, tag VARCHAR(100) DEFAULT NULL, follows VARCHAR(20) DEFAULT NULL, fans VARCHAR(20) DEFAULT NULL, posts VARCHAR(20) DEFAULT NULL)'
        
        cur.execute(sqldrop)
        cur.execute(sqlcreate)
        
        conn.commit()
        
    except:
        conn.rollback()
        
    finally:
        if conn:
            conn.close()
        if cur:
            cur.close()

In [27]:
def connMySQL():
    conn = pymysql.connect('localhost', 'msba', 'hku2019', 'makeup')
    conn.autocommit(1)
    cur = conn.cursor()
    return cur

In [28]:
def find_bloggers(username, password, topic):
    
    driver = webdriver.Firefox()
    driver.get('http://weibo.com/login.php')
    
    driver.find_element_by_name('username').send_keys(username)
    driver.find_element_by_name('password').send_keys(password)
    time.sleep(5) # time to input validation code
    
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/div/div/div[3]/div[6]').click()
    time.sleep(5)
    
    driver.get('https://s.weibo.com/')
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/a[2]').click()
    time.sleep(5)
    
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').clear()
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').send_keys(topic)
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/button').click()
    time.sleep(5)

    ele = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span')
    ActionChains(driver).move_to_element(ele).perform()
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span/ul/li[3]/a').click()
    time.sleep(5)
    
    data = {}
    
    for j in tqdm(range(1, 44)):
    
        for i in tqdm(range(len(driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/*')))):
            
            table = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/div['+str(i+1)+']/div[2]')
            
            if table.find_element_by_xpath('p[3]').text[:2] != '关注':
                continue
            
            name = table.find_element_by_xpath('div/a[1]').text    
            uid = table.find_element_by_xpath('p[3]/span[1]/a').get_attribute('href')
            uid = re.findall('[0-9]+', uid)[0]
            gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
            gender = re.findall('icon-sex-(.*)', gender)[0]
            region = table.find_element_by_xpath('p[1]').text.split(' ')[0]
            
            if len(table.find_elements_by_xpath('p[2]/*')) == 0:
                tag = table.find_element_by_xpath('p[2]').text.split(' ')
                follows = table.find_element_by_xpath('p[3]/span/a').text
                fans = table.find_element_by_xpath('p[3]/span[2]/a').text
                posts = table.find_element_by_xpath('p[3]/span[3]/a').text
            else:
                tag = []
                follows = table.find_element_by_xpath('p[2]/span/a').text
                fans = table.find_element_by_xpath('p[2]/span[2]/a').text
                posts = table.find_element_by_xpath('p[2]/span[3]/a').text
                
            data[len(data)] = {'name':name, 'uid':uid, 'gender':gender,'region':region,
                               'tag':tag,'follows':follows,'fans':fans,'posts':posts}
            
        time.sleep(5)
       
        if j != 43:
            driver.find_element_by_class_name('next').click()
        
        time.sleep(5)

            
    return data

In [41]:
def dict_to_df(data):
    
    df = pd.DataFrame(data).T
    cols = ['name', 'uid', 'gender', 'region', 'tag', 'follows', 'fans', 'posts']
    df = df.loc[:,cols]
    
    for i in range(len(df)):
        
        df['tag'][i] = ', '.join(df['tag'][i])
        
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['follows'][i])):
            df['follows'][i] = df['follows'][i][:-1] + '0000'
        else:
             df['follows'][i] = df['follows'][i]
        
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['fans'][i])):
            df['fans'][i] = df['fans'][i][:-1] + '0000'
        else:
             df['fans'][i] = df['fans'][i]
                
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['posts'][i])):
            df['posts'][i] = df['posts'][i][:-1] + '0000'
        else:
             df['posts'][i] = df['posts'][i]
                
    return df

## Keyword 1: 美妆自媒体

In [7]:
data1 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '美妆自媒体')

ElementNotInteractableException: Message: Element <input name="username" type="hidden"> is not reachable by keyboard


In [9]:
df1 = dict_to_df(data1)

In [10]:
df1

,name,uid,gender,region,tag,follows,fans,posts
0,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,全球时尚,1660209951,female,海外,"微博知名时尚帐号, 时尚美妆视频自媒体",193,14690000,90000
2,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
3,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
4,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
5,子望,2395743084,female,其他,"演员, 作家, 时尚美妆视频自媒体",128,3870000,837
6,虫虫Chonny,2634154091,female,浙江,"时尚达人, 知名美妆博主, 微博签约自媒体",560,4610000,916
7,-Fashionbaby,1765813640,female,广东,"时尚达人, 知名时尚博主, 时尚美妆视频自媒体",698,3740000,10000
8,管阿姨,1225512093,female,浙江,"微博vlog博主, 杭州柳泉文化创意有限公司, 摄影部总监, 知名时尚博主, 时尚美妆视频自媒体",1182,2510000,7702
9,高颜值集中营,5596017363,female,北京,"时尚博主, 时尚美妆视频自媒体",125,2620000,6726


In [79]:
df1.to_csv('bloggers1.csv', encoding = 'utf_8_sig')

## Keyword 2: 时尚自媒体

In [43]:
data2 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '时尚自媒体')






  0%|          | 0/50 [00:00<?, ?it/s]




  2%|▏         | 1/50 [00:32<26:42, 32.71s/it]




  4%|▍         | 2/50 [01:05<26:12, 32.75s/it]




  6%|▌         | 3/50 [01:38<25:47, 32.93s/it]




  8%|▊         | 4/50 [02:11<25:14, 32.92s/it]




 10%|█         | 5/50 [02:44<24:33, 32.74s/it]




 12%|█▏        | 6/50 [03:16<23:56, 32.65s/it]




 14%|█▍        | 7/50 [03:49<23:22, 32.61s/it]




 16%|█▌        | 8/50 [04:21<22:45, 32.51s/it]




 18%|█▊        | 9/50 [04:53<22:11, 32.48s/it]




 20%|██        | 10/50 [05:26<21:41, 32.54s/it]




 22%|██▏       | 11/50 [05:59<21:15, 32.70s/it]




 24%|██▍       | 12/50 [06:33<20:53, 32.98s/it]




 26%|██▌       | 13/50 [07:05<20:17, 32.90s/it]




 28%|██▊       | 14/50 [07:38<19:38, 32.73s/it]




 30%|███       | 15/50 [08:10<19:05, 32.72s/it]




 32%|███▏      | 16/50 [08:44<18:38, 32.91s/it]




 34%|███▍      | 17/50 [09:17<18:07, 32.96s/it]




 36%|███▌      | 18/50 [09:50<17:40, 33.15s/it]




 38%|███▊      | 19/50 [1

In [44]:
df2 = dict_to_df(data2)

In [45]:
df2

,name,uid,gender,region,tag,follows,fans,posts
0,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8920000,909
1,gogoboi,1706372681,male,上海,"时尚达人, 时装专栏作者, 微博签约自媒体",516,9850000,9075
2,全球时尚,1660209951,female,海外,"微博知名时尚帐号, 时尚美妆视频自媒体",193,14690000,90000
3,丁一晨DYC,1810632930,female,北京,"丁小点形象作者, 微博vlog博主, 知名时尚博主, 萌宠视频自媒体, 微博签约自媒体",1398,7860000,6545
4,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
5,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
6,CNU_blank,1816029971,male,广东,"中国视觉联盟（CNU）时尚、摄影编辑，摄影师, 微博签约自媒体",155,8080000,20000
7,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
8,买买菌,5098547539,female,北京,"时尚达人, 微博知名博主, 微博原创视频博主, 微博签约自媒体",1324,4060000,10000
9,私服街拍,2170479940,female,北京,"知名时尚博主, 微博签约自媒体",546,8660000,10000


In [85]:
df2.to_csv('bloggers2.csv', encoding = 'utf_8_sig')

## Keyword 3: 美妆达人

In [76]:
data3 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '美妆达人')






  0%|          | 0/50 [00:00<?, ?it/s]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  6.75it/s]





 10%|█         | 2/20 [00:00<00:02,  6.92it/s]





 15%|█▌        | 3/20 [00:00<00:02,  7.35it/s]





 20%|██        | 4/20 [00:00<00:02,  7.67it/s]





 25%|██▌       | 5/20 [00:00<00:01,  7.98it/s]





 30%|███       | 6/20 [00:00<00:01,  8.02it/s]





 35%|███▌      | 7/20 [00:00<00:01,  8.28it/s]





 40%|████      | 8/20 [00:00<00:01,  8.49it/s]





 45%|████▌     | 9/20 [00:01<00:01,  8.48it/s]





 50%|█████     | 10/20 [00:01<00:01,  8.46it/s]





 55%|█████▌    | 11/20 [00:01<00:01,  8.00it/s]





 60%|██████    | 12/20 [00:01<00:01,  7.88it/s]





 65%|██████▌   | 13/20 [00:01<00:00,  7.55it/s]





 70%|███████   | 14/20 [00:01<00:00,  7.51it/s]





 75%|███████▌  | 15/20 [00:01<00:00,  7.91it/s]





 80%|████████  | 16/20 [00:02<00:00,  7.80it/s]





 85%|████████▌ | 17/20 [00:02<00:00,  7.92it/s]





 90%|█████████ 

 24%|██▍       | 12/50 [07:46<24:22, 38.49s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  9.10it/s]





 10%|█         | 2/20 [00:00<00:01,  9.33it/s]





 20%|██        | 4/20 [00:00<00:01,  9.76it/s]





 30%|███       | 6/20 [00:00<00:01, 10.02it/s]





 40%|████      | 8/20 [00:00<00:01, 10.32it/s]





 50%|█████     | 10/20 [00:00<00:00, 10.58it/s]





 55%|█████▌    | 11/20 [00:01<00:00, 10.40it/s]





 65%|██████▌   | 13/20 [00:01<00:00, 10.59it/s]





 75%|███████▌  | 15/20 [00:01<00:00, 10.64it/s]





 85%|████████▌ | 17/20 [00:01<00:00, 10.70it/s]





 95%|█████████▌| 19/20 [00:01<00:00, 10.81it/s]





100%|██████████| 20/20 [00:01<00:00, 10.72it/s]




 26%|██▌       | 13/50 [08:25<23:49, 38.62s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  9.00it/s]





 15%|█▌        | 3/20 [00:00<00:01,  9.49it/s]





 25%|██▌       | 5/20 [00:00<00:01,  9.80it/s]





 35%|███▌      | 7/2

 50%|█████     | 10/20 [00:01<00:01,  8.37it/s]





 55%|█████▌    | 11/20 [00:01<00:01,  8.71it/s]





 60%|██████    | 12/20 [00:01<00:00,  8.88it/s]





 65%|██████▌   | 13/20 [00:01<00:00,  9.05it/s]





 70%|███████   | 14/20 [00:01<00:00,  6.21it/s]





 75%|███████▌  | 15/20 [00:02<00:00,  6.05it/s]





 80%|████████  | 16/20 [00:02<00:00,  6.55it/s]





 85%|████████▌ | 17/20 [00:02<00:00,  7.14it/s]





 90%|█████████ | 18/20 [00:02<00:00,  7.59it/s]





 95%|█████████▌| 19/20 [00:02<00:00,  7.98it/s]





100%|██████████| 20/20 [00:02<00:00,  8.47it/s]




 54%|█████▍    | 27/50 [17:28<14:55, 38.96s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  8.17it/s]





 15%|█▌        | 3/20 [00:00<00:01,  8.67it/s]





 20%|██        | 4/20 [00:00<00:01,  8.93it/s]





 25%|██▌       | 5/20 [00:00<00:02,  7.12it/s]





 30%|███       | 6/20 [00:00<00:01,  7.55it/s]





 35%|███▌      | 7/20 [00:00<00:01,  7.42it/s]





 40%|████

  5%|▌         | 1/20 [00:00<00:02,  9.02it/s]





 10%|█         | 2/20 [00:00<00:01,  9.17it/s]





 15%|█▌        | 3/20 [00:00<00:01,  9.41it/s]





 25%|██▌       | 5/20 [00:00<00:01,  9.77it/s]





 35%|███▌      | 7/20 [00:00<00:01,  9.97it/s]





 45%|████▌     | 9/20 [00:00<00:01, 10.28it/s]





 55%|█████▌    | 11/20 [00:01<00:00, 10.48it/s]





 65%|██████▌   | 13/20 [00:01<00:00, 10.56it/s]





 75%|███████▌  | 15/20 [00:01<00:00, 10.63it/s]





 85%|████████▌ | 17/20 [00:01<00:00, 10.75it/s]





 95%|█████████▌| 19/20 [00:01<00:00, 10.82it/s]





100%|██████████| 20/20 [00:01<00:00, 10.61it/s]




 80%|████████  | 40/50 [25:58<06:32, 39.27s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  8.89it/s]





 10%|█         | 2/20 [00:00<00:01,  9.02it/s]





 20%|██        | 4/20 [00:00<00:01,  9.52it/s]





 30%|███       | 6/20 [00:00<00:01,  9.87it/s]





 40%|████      | 8/20 [00:00<00:01, 10.21it/s]





 50%|█████    

In [106]:
df3 = dict_to_df(data3)

In [107]:
df3

,name,uid,gender,region,tag,follows,fans,posts
0,美妆达人馆,6042105257,male,北京,微博时尚美妆视频帐号,96,500000,20000
1,美妆达人菌,5712824035,female,北京,知名美妆博主,28,1160000,509
2,美粧达人,5884054391,female,广东,美妆博主,147,90000,248
3,美妆种草达人,5225142517,male,海外,知名美妆博主,42,130000,160000
4,美妆达人小美,6355361363,female,江苏,时尚博主,76,120000,110000
5,美妆白菜达人,5774414934,female,江苏,知名美妆博主,53,120000,160000
6,美妆囤货达人,5288563242,female,湖北,美妆博主,282,130000,80000
7,美妆达人大王,5774391783,male,福建,美妆博主,306,100000,80000
8,是美妆小达人,5292500369,female,山东,美妆博主,174,70000,20000
9,美妆用品达人,5774384913,female,江西,美妆博主,288,70000,90000


In [89]:
df3.to_csv('bloggers3.csv', encoding = 'utf_8_sig')

## Keyword 4: 时尚达人

In [85]:
data4 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '时尚达人')






  0%|          | 0/50 [00:00<?, ?it/s]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  6.96it/s]





 10%|█         | 2/20 [00:00<00:02,  7.27it/s]





 15%|█▌        | 3/20 [00:00<00:02,  7.59it/s]





 20%|██        | 4/20 [00:00<00:02,  7.91it/s]





 25%|██▌       | 5/20 [00:00<00:01,  8.18it/s]





 30%|███       | 6/20 [00:00<00:01,  8.16it/s]





 35%|███▌      | 7/20 [00:00<00:01,  8.39it/s]





 40%|████      | 8/20 [00:00<00:01,  8.43it/s]





 45%|████▌     | 9/20 [00:01<00:01,  8.54it/s]





 50%|█████     | 10/20 [00:01<00:01,  8.54it/s]





 55%|█████▌    | 11/20 [00:01<00:01,  8.44it/s]





 60%|██████    | 12/20 [00:01<00:00,  8.53it/s]





 65%|██████▌   | 13/20 [00:01<00:00,  8.51it/s]





 70%|███████   | 14/20 [00:01<00:00,  8.65it/s]





 75%|███████▌  | 15/20 [00:01<00:00,  8.65it/s]





 80%|████████  | 16/20 [00:01<00:00,  8.54it/s]





 85%|████████▌ | 17/20 [00:02<00:00,  8.55it/s]





 90%|█████████ 

 25%|██▌       | 5/20 [00:00<00:01,  9.21it/s]





 30%|███       | 6/20 [00:00<00:01,  9.25it/s]





 40%|████      | 8/20 [00:00<00:01,  9.64it/s]





 45%|████▌     | 9/20 [00:00<00:01,  9.73it/s]





 55%|█████▌    | 11/20 [00:01<00:00,  9.90it/s]





 60%|██████    | 12/20 [00:01<00:00,  9.88it/s]





 70%|███████   | 14/20 [00:01<00:00, 10.08it/s]





 75%|███████▌  | 15/20 [00:01<00:00, 10.01it/s]





 80%|████████  | 16/20 [00:01<00:00,  9.84it/s]





 90%|█████████ | 18/20 [00:01<00:00, 10.08it/s]





100%|██████████| 20/20 [00:02<00:00, 10.21it/s]




 26%|██▌       | 13/50 [08:20<23:39, 38.38s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  9.04it/s]





 10%|█         | 2/20 [00:00<00:01,  9.17it/s]





 15%|█▌        | 3/20 [00:00<00:01,  9.28it/s]





 20%|██        | 4/20 [00:00<00:01,  9.48it/s]





 30%|███       | 6/20 [00:00<00:01,  9.82it/s]





 40%|████      | 8/20 [00:00<00:01, 10.16it/s]





 50%|█████   

 45%|████▌     | 9/20 [00:01<00:01,  8.55it/s]





 50%|█████     | 10/20 [00:01<00:01,  8.60it/s]





 55%|█████▌    | 11/20 [00:01<00:01,  8.46it/s]





 60%|██████    | 12/20 [00:01<00:00,  8.47it/s]





 65%|██████▌   | 13/20 [00:01<00:00,  8.48it/s]





 70%|███████   | 14/20 [00:01<00:00,  8.59it/s]





 75%|███████▌  | 15/20 [00:01<00:00,  8.62it/s]





 80%|████████  | 16/20 [00:01<00:00,  8.49it/s]





 85%|████████▌ | 17/20 [00:02<00:00,  8.55it/s]





 90%|█████████ | 18/20 [00:02<00:00,  8.55it/s]





 95%|█████████▌| 19/20 [00:02<00:00,  8.56it/s]





100%|██████████| 20/20 [00:02<00:00,  8.58it/s]




 54%|█████▍    | 27/50 [17:16<14:38, 38.21s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  9.27it/s]





 15%|█▌        | 3/20 [00:00<00:01,  9.64it/s]





 20%|██        | 4/20 [00:00<00:01,  9.70it/s]





 25%|██▌       | 5/20 [00:00<00:01,  9.65it/s]





 35%|███▌      | 7/20 [00:00<00:01,  9.95it/s]





 45%|████

 10%|█         | 2/20 [00:00<00:02,  8.60it/s]





 15%|█▌        | 3/20 [00:00<00:01,  8.88it/s]





 20%|██        | 4/20 [00:00<00:01,  9.18it/s]





 30%|███       | 6/20 [00:00<00:01,  9.49it/s]





 40%|████      | 8/20 [00:00<00:01,  9.81it/s]





 50%|█████     | 10/20 [00:00<00:00, 10.09it/s]





 55%|█████▌    | 11/20 [00:01<00:00,  9.94it/s]





 60%|██████    | 12/20 [00:01<00:00,  9.90it/s]





 65%|██████▌   | 13/20 [00:01<00:00,  9.89it/s]





 75%|███████▌  | 15/20 [00:01<00:00, 10.08it/s]





 80%|████████  | 16/20 [00:01<00:00,  9.96it/s]





 90%|█████████ | 18/20 [00:01<00:00, 10.15it/s]





100%|██████████| 20/20 [00:01<00:00, 10.26it/s]




 82%|████████▏ | 41/50 [26:08<05:41, 37.92s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  8.13it/s]





 10%|█         | 2/20 [00:00<00:02,  8.55it/s]





 20%|██        | 4/20 [00:00<00:01,  9.14it/s]





 25%|██▌       | 5/20 [00:00<00:01,  9.30it/s]





 35%|███▌   

In [108]:
df4 = dict_to_df(data4)

In [109]:
df4

,name,uid,gender,region,tag,follows,fans,posts
0,时尚达人Simo,1794845531,female,北京,"时尚达人, 北京《瑞丽》杂志社, 时尚, 特约撰稿人, 知名时尚博主",82,7320000,20000
1,时尚达人Danny,6027416124,male,广东,搞笑幽默博主,133,1040000,208
2,时尙达人,5263923753,female,海外,时尚达人,57,10000,3765
3,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8940000,909
4,Pony__朴惠敏,5882998192,female,海外,"时尚达人，MUNMU, INC., 签约美妆艺术家，资深美妆达人",13,10020000,830
5,时尚达人吖,1771926912,female,台湾,台湾名模陈思璇,201,60000,362
6,Kakakaoo-,1927564525,female,上海,"微博原创视频博主, 时尚达人",379,13200000,2299
7,gogoboi,1706372681,male,上海,"时尚达人, 时装专栏作者, 微博签约自媒体",516,9850000,9075
8,呛口小辣椒,1732235143,female,其他,"时尚达人, 知名时尚博主, 微电商达人",134,9970000,3598
9,韩火火,1263585717,male,北京,时尚达人，Stylist,791,12400000,4159


In [93]:
df4.to_csv('bloggers4.csv', encoding = 'utf_8_sig')

## Keyword 5: 美妆博主

In [23]:
data5 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '美妆博主')




  0%|          | 0/50 [00:00<?, ?it/s]



  0%|          | 0/20 [00:00<?, ?it/s]



  5%|▌         | 1/20 [00:00<00:02,  6.54it/s]



 10%|█         | 2/20 [00:00<00:02,  6.89it/s]



 15%|█▌        | 3/20 [00:00<00:02,  7.13it/s]



 20%|██        | 4/20 [00:00<00:02,  7.45it/s]



 25%|██▌       | 5/20 [00:00<00:02,  7.32it/s]



 30%|███       | 6/20 [00:00<00:02,  6.59it/s]



 35%|███▌      | 7/20 [00:00<00:01,  7.08it/s]



 40%|████      | 8/20 [00:01<00:01,  7.50it/s]



 45%|████▌     | 9/20 [00:01<00:01,  7.85it/s]



 50%|█████     | 10/20 [00:01<00:01,  8.08it/s]



 55%|█████▌    | 11/20 [00:01<00:01,  8.12it/s]



 60%|██████    | 12/20 [00:01<00:00,  8.30it/s]



 65%|██████▌   | 13/20 [00:01<00:00,  8.52it/s]



 70%|███████   | 14/20 [00:01<00:00,  8.44it/s]



 75%|███████▌  | 15/20 [00:01<00:00,  8.55it/s]



 80%|████████  | 16/20 [00:02<00:00,  8.48it/s]



 85%|████████▌ | 17/20 [00:02<00:00,  8.59it/s]



 90%|█████████ | 18/20 [00:02<00:00,  8.67it/s]



 95%

KeyboardInterrupt: 

In [119]:
df5 = dict_to_df(data5)

In [120]:
df5

,name,uid,gender,region,tag,follows,fans,posts
0,美粧博主,5881204733,female,其他,知名美妆博主,149,870000,4239
1,美妆博主,5322214912,female,海外,美妆博主,1281,70000,92
2,美妆博主推荐,6019698320,female,江苏,"知名美妆博主, 时尚美妆视频自媒体",153,1700000,20000
3,英國時尚美妆博主LIZ,1743031157,female,海外,"时尚博主, 时尚美妆视频自媒体",457,860000,2074
4,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8940000,909
5,美妆博主日记,5763694930,female,香港,"知名美妆博主, 时尚美妆视频自媒体",73,540000,6599
6,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
7,阿花花酱,1695958733,female,江苏,"时尚达人, 知名美妆博主",682,6300000,7054
8,种草小星星,6571964646,female,广东,知名美妆博主,49,3880000,60000
9,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1015


In [98]:
df5.to_csv('bloggers5.csv', encoding = 'utf_8_sig')

## Keyword 6: 时尚博主

In [27]:
data6 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '时尚博主')






  0%|          | 0/50 [00:00<?, ?it/s]





  0%|          | 0/20 [00:00<?, ?it/s]





 30%|███       | 6/20 [00:00<00:00, 55.31it/s]





 40%|████      | 8/20 [00:00<00:00, 30.95it/s]





 80%|████████  | 16/20 [00:00<00:00, 33.07it/s]





 95%|█████████▌| 19/20 [00:00<00:00, 28.70it/s]





100%|██████████| 20/20 [00:00<00:00, 28.87it/s]




  2%|▏         | 1/50 [00:41<33:51, 41.46s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  8.61it/s]





 10%|█         | 2/20 [00:00<00:02,  8.84it/s]





 15%|█▌        | 3/20 [00:00<00:01,  9.06it/s]





 20%|██        | 4/20 [00:00<00:01,  8.10it/s]





 25%|██▌       | 5/20 [00:00<00:01,  8.39it/s]





 30%|███       | 6/20 [00:00<00:01,  8.69it/s]





 35%|███▌      | 7/20 [00:00<00:01,  8.80it/s]





 40%|████      | 8/20 [00:00<00:01,  9.05it/s]





 45%|████▌     | 9/20 [00:01<00:01,  9.05it/s]





 50%|█████     | 10/20 [00:01<00:01,  9.22it/s]





 55%|█████▌    | 11/20 [00:0

 50%|█████     | 10/20 [00:01<00:01,  9.41it/s]





 55%|█████▌    | 11/20 [00:01<00:00,  9.40it/s]





 60%|██████    | 12/20 [00:01<00:00,  9.50it/s]





 65%|██████▌   | 13/20 [00:01<00:00,  9.64it/s]





 75%|███████▌  | 15/20 [00:01<00:00,  9.89it/s]





 80%|████████  | 16/20 [00:01<00:00,  9.86it/s]





 85%|████████▌ | 17/20 [00:01<00:00,  9.83it/s]





 90%|█████████ | 18/20 [00:01<00:00,  9.33it/s]





 95%|█████████▌| 19/20 [00:02<00:00,  9.42it/s]





100%|██████████| 20/20 [00:02<00:00,  9.50it/s]




 28%|██▊       | 14/50 [09:12<23:20, 38.91s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  8.85it/s]





 10%|█         | 2/20 [00:00<00:02,  8.93it/s]





 20%|██        | 4/20 [00:00<00:01,  9.39it/s]





 25%|██▌       | 5/20 [00:00<00:01,  9.44it/s]





 30%|███       | 6/20 [00:00<00:01,  9.54it/s]





 35%|███▌      | 7/20 [00:00<00:01,  9.66it/s]





 45%|████▌     | 9/20 [00:00<00:01,  9.98it/s]





 55%|█████

 50%|█████     | 10/20 [00:01<00:01,  9.14it/s]





 55%|█████▌    | 11/20 [00:01<00:00,  9.11it/s]





 60%|██████    | 12/20 [00:01<00:00,  9.21it/s]





 65%|██████▌   | 13/20 [00:01<00:00,  9.26it/s]





 70%|███████   | 14/20 [00:01<00:00,  9.39it/s]





 75%|███████▌  | 15/20 [00:01<00:00,  9.44it/s]





 80%|████████  | 16/20 [00:01<00:00,  9.26it/s]





 85%|████████▌ | 17/20 [00:01<00:00,  9.36it/s]





 90%|█████████ | 18/20 [00:01<00:00,  9.27it/s]





 95%|█████████▌| 19/20 [00:02<00:00,  9.30it/s]





100%|██████████| 20/20 [00:02<00:00,  9.34it/s]




 50%|█████     | 25/50 [16:24<16:17, 39.11s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  8.66it/s]





 10%|█         | 2/20 [00:00<00:02,  8.83it/s]





 15%|█▌        | 3/20 [00:00<00:01,  9.01it/s]





 20%|██        | 4/20 [00:00<00:01,  9.27it/s]





 25%|██▌       | 5/20 [00:00<00:01,  9.41it/s]





 30%|███       | 6/20 [00:00<00:01,  9.49it/s]





 35%|███▌

 53%|█████▎    | 10/19 [00:01<00:00,  9.63it/s]





 58%|█████▊    | 11/19 [00:01<00:00,  9.49it/s]





 63%|██████▎   | 12/19 [00:01<00:00,  9.50it/s]





 68%|██████▊   | 13/19 [00:01<00:00,  9.54it/s]





 74%|███████▎  | 14/19 [00:01<00:00,  9.65it/s]





 79%|███████▉  | 15/19 [00:01<00:00,  9.70it/s]





 84%|████████▍ | 16/19 [00:01<00:00,  9.55it/s]





 89%|████████▉ | 17/19 [00:01<00:00,  9.53it/s]





 95%|█████████▍| 18/19 [00:01<00:00,  9.50it/s]





100%|██████████| 19/19 [00:02<00:00,  9.48it/s]




 74%|███████▍  | 37/50 [24:12<08:17, 38.27s/it]





  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [00:00<00:02,  8.68it/s]





 10%|█         | 2/20 [00:00<00:02,  8.87it/s]





 15%|█▌        | 3/20 [00:00<00:01,  9.03it/s]





 20%|██        | 4/20 [00:00<00:01,  9.26it/s]





 25%|██▌       | 5/20 [00:00<00:01,  9.43it/s]





 30%|███       | 6/20 [00:00<00:01,  9.57it/s]





 35%|███▌      | 7/20 [00:00<00:01,  9.69it/s]





 45%|████▌

 39%|███▉      | 7/18 [00:00<00:01,  9.34it/s]





 44%|████▍     | 8/18 [00:00<00:01,  9.49it/s]





 50%|█████     | 9/18 [00:00<00:00,  9.52it/s]





 56%|█████▌    | 10/18 [00:01<00:00,  9.58it/s]





 61%|██████    | 11/18 [00:01<00:00,  9.50it/s]





 67%|██████▋   | 12/18 [00:01<00:00,  9.52it/s]





 72%|███████▏  | 13/18 [00:01<00:00,  9.50it/s]





 78%|███████▊  | 14/18 [00:01<00:00,  9.60it/s]





 83%|████████▎ | 15/18 [00:01<00:00,  9.65it/s]





 89%|████████▉ | 16/18 [00:01<00:00,  9.51it/s]





 94%|█████████▍| 17/18 [00:01<00:00,  9.57it/s]





100%|██████████| 18/18 [00:01<00:00,  9.62it/s]




 98%|█████████▊| 49/50 [31:51<00:38, 38.05s/it]





  0%|          | 0/19 [00:00<?, ?it/s]





  5%|▌         | 1/19 [00:00<00:02,  8.56it/s]





 11%|█         | 2/19 [00:00<00:01,  8.68it/s]





 16%|█▌        | 3/19 [00:00<00:01,  8.97it/s]





 26%|██▋       | 5/19 [00:00<00:01,  9.38it/s]





 32%|███▏      | 6/19 [00:00<00:01,  9.51it/s]





 37%|███▋  

In [28]:
df6 = dict_to_df(data6)

In [29]:
df6

,name,uid,gender,region,tag,follows,fans,posts
0,英國時尚美妆博主LIZ,1743031157,female,海外,"时尚博主, 时尚美妆视频自媒体",457,860000,2074
1,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",283,8950000,910
2,Kakakaoo-,1927564525,female,上海,"微博原创视频博主, 时尚达人",379,13210000,2299
3,时尚博主-Rose女士,1822046155,female,北京,", , 北京逸槿国际商贸有限公司Rose, paris首席设计师夏梦莹",379,380000,796
4,买买菌,5098547539,female,北京,"时尚达人, 微博知名博主, 微博原创视频博主, 微博签约自媒体",1324,4070000,10000
5,andy_bw,6066271469,male,北京,知名时尚博主,52,8220000,218
6,美少女Lisa酱,5041507691,female,北京,"时尚达人, 微博vlog博主",327,3390000,3503
7,买买小天使,3191534861,female,广东,知名时尚博主,837,6080000,7758
8,私服街拍,2170479940,female,北京,"知名时尚博主, 微博签约自媒体",545,8660000,10000
9,头像集,2681064973,female,其他,"时尚达人, 知名情感博主",152,8680000,70000


In [102]:
df6.to_csv('bloggers6.csv', encoding = 'utf_8_sig')

## Keyword 7: 微博美妆博主

In [36]:
data7 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '微博美妆博主')








  0%|          | 0/43 [00:00<?, ?it/s]







  0%|          | 0/20 [00:00<?, ?it/s]







  5%|▌         | 1/20 [00:00<00:02,  7.38it/s]







 10%|█         | 2/20 [00:00<00:02,  7.78it/s]







 15%|█▌        | 3/20 [00:00<00:02,  8.17it/s]







 20%|██        | 4/20 [00:00<00:01,  8.46it/s]







 25%|██▌       | 5/20 [00:00<00:01,  8.70it/s]







 30%|███       | 6/20 [00:00<00:01,  8.57it/s]







 35%|███▌      | 7/20 [00:00<00:01,  8.57it/s]







 40%|████      | 8/20 [00:00<00:01,  8.66it/s]







 45%|████▌     | 9/20 [00:01<00:01,  8.88it/s]







 50%|█████     | 10/20 [00:01<00:01,  8.95it/s]







 55%|█████▌    | 11/20 [00:01<00:01,  8.99it/s]







 60%|██████    | 12/20 [00:01<00:00,  9.14it/s]







 65%|██████▌   | 13/20 [00:01<00:00,  9.16it/s]







 70%|███████   | 14/20 [00:01<00:00,  9.33it/s]







 75%|███████▌  | 15/20 [00:01<00:00,  9.36it/s]







 80%|████████  | 16/20 [00:01<00:00,  9.20it/s]







 85%|████████▌ | 17/20 [00:01<

 65%|██████▌   | 13/20 [00:01<00:01,  6.58it/s]







 70%|███████   | 14/20 [00:01<00:00,  6.51it/s]







 75%|███████▌  | 15/20 [00:02<00:00,  6.44it/s]







 80%|████████  | 16/20 [00:02<00:00,  7.00it/s]







 85%|████████▌ | 17/20 [00:02<00:00,  6.75it/s]







 90%|█████████ | 18/20 [00:02<00:00,  6.57it/s]







 95%|█████████▌| 19/20 [00:02<00:00,  7.15it/s]







100%|██████████| 20/20 [00:02<00:00,  7.24it/s]







 28%|██▊       | 12/43 [02:35<06:42, 12.98s/it]







  0%|          | 0/20 [00:00<?, ?it/s]







  5%|▌         | 1/20 [00:00<00:02,  8.57it/s]







 10%|█         | 2/20 [00:00<00:02,  8.90it/s]







 20%|██        | 4/20 [00:00<00:01,  9.41it/s]







 30%|███       | 6/20 [00:00<00:01,  9.67it/s]







 40%|████      | 8/20 [00:00<00:01, 10.05it/s]







 50%|█████     | 10/20 [00:00<00:00, 10.34it/s]







 55%|█████▌    | 11/20 [00:01<00:00, 10.16it/s]







 65%|██████▌   | 13/20 [00:01<00:00, 10.33it/s]







 70%|███████   | 14/20 [

 85%|████████▌ | 17/20 [00:01<00:00,  9.19it/s]







 90%|█████████ | 18/20 [00:02<00:00,  9.26it/s]







 95%|█████████▌| 19/20 [00:02<00:00,  9.34it/s]







100%|██████████| 20/20 [00:02<00:00,  8.83it/s]







 56%|█████▌    | 24/43 [05:08<04:06, 12.97s/it]







  0%|          | 0/20 [00:00<?, ?it/s]







  5%|▌         | 1/20 [00:00<00:02,  8.44it/s]







 10%|█         | 2/20 [00:00<00:02,  8.75it/s]







 20%|██        | 4/20 [00:00<00:01,  9.28it/s]







 30%|███       | 6/20 [00:00<00:01,  9.60it/s]







 40%|████      | 8/20 [00:00<00:01,  9.94it/s]







 45%|████▌     | 9/20 [00:00<00:01,  9.92it/s]







 50%|█████     | 10/20 [00:00<00:01,  9.80it/s]







 55%|█████▌    | 11/20 [00:01<00:00,  9.84it/s]







 65%|██████▌   | 13/20 [00:01<00:00, 10.10it/s]







 75%|███████▌  | 15/20 [00:01<00:00, 10.36it/s]







 85%|████████▌ | 17/20 [00:01<00:00, 10.43it/s]







100%|██████████| 20/20 [00:01<00:00, 10.35it/s]







 58%|█████▊    | 25/43 [0

 67%|██████▋   | 12/18 [00:01<00:00,  9.60it/s]







 72%|███████▏  | 13/18 [00:01<00:00,  8.70it/s]







 83%|████████▎ | 15/18 [00:01<00:00,  9.17it/s]







 89%|████████▉ | 16/18 [00:01<00:00,  9.34it/s]







100%|██████████| 18/18 [00:01<00:00,  9.66it/s]







 81%|████████▏ | 35/43 [07:30<01:42, 12.83s/it]







  0%|          | 0/20 [00:00<?, ?it/s]







  5%|▌         | 1/20 [00:00<00:02,  8.57it/s]







 10%|█         | 2/20 [00:00<00:02,  8.90it/s]







 15%|█▌        | 3/20 [00:00<00:01,  9.16it/s]







 25%|██▌       | 5/20 [00:00<00:01,  9.65it/s]







 35%|███▌      | 7/20 [00:00<00:01, 10.03it/s]







 45%|████▌     | 9/20 [00:00<00:01, 10.46it/s]







 55%|█████▌    | 11/20 [00:01<00:00, 10.65it/s]







 65%|██████▌   | 13/20 [00:01<00:00, 10.64it/s]







 75%|███████▌  | 15/20 [00:01<00:00, 10.83it/s]







 85%|████████▌ | 17/20 [00:01<00:00, 10.76it/s]







100%|██████████| 20/20 [00:01<00:00, 10.73it/s]







 84%|████████▎ | 36/43 [0

In [48]:
df7 = dict_to_df(data7)

In [38]:
df7

,name,uid,gender,region,tag,follows,fans,posts
0,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
1,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5700000,1017
2,虫虫Chonny,2634154091,female,浙江,"时尚达人, 知名美妆博主, 微博签约自媒体",560,4610000,919
3,扇子NO_FAN_NO_FUN,1944603171,female,海外,"微博vlog博主, 快美妆签约红人, 时尚达人, 知名美妆博主",162,2760000,1530
4,管阿姨,1225512093,female,浙江,"微博vlog博主, 杭州柳泉文化创意有限公司, 摄影部总监, 知名时尚博主, 时尚美妆视频自媒体",1182,2510000,7703
...,...,...,...,...,...,...,...,...
846,南京无痕接发苏洋,5081701352,male,江苏,时尚美妆视频自媒体,145,2625,117
847,Vlogger小V,6120686578,female,北京,微博vlog博主,208,2409,89
848,美容护肤师_娜娜,3805931144,female,山西,太原伊诗美商贸有限公司营养护肤老师,986,1776,778
849,陆衫定z7m0,6329772692,female,贵州,美妆博主,517,416,668


In [56]:
df7.to_csv('bloggers7.csv', encoding = 'utf_8_sig')

## Keyword 8: 微博时尚博主

In [120]:
data8 = find_bloggers('lanjiao46505@163.com', 'asd123123a', '微博时尚博主')







  0%|          | 0/50 [00:00<?, ?it/s]






  0%|          | 0/20 [00:00<?, ?it/s]






  5%|▌         | 1/20 [00:00<00:05,  3.41it/s]






 10%|█         | 2/20 [00:00<00:04,  4.09it/s]






 15%|█▌        | 3/20 [00:00<00:04,  4.23it/s]






 20%|██        | 4/20 [00:00<00:03,  4.39it/s]






 25%|██▌       | 5/20 [00:00<00:03,  5.00it/s]






 30%|███       | 6/20 [00:01<00:02,  5.62it/s]






 35%|███▌      | 7/20 [00:01<00:02,  6.24it/s]






 40%|████      | 8/20 [00:01<00:01,  6.33it/s]






 45%|████▌     | 9/20 [00:01<00:01,  5.68it/s]






 50%|█████     | 10/20 [00:01<00:01,  6.10it/s]






 55%|█████▌    | 11/20 [00:01<00:01,  6.28it/s]






 60%|██████    | 12/20 [00:02<00:01,  6.43it/s]






 65%|██████▌   | 13/20 [00:02<00:01,  6.73it/s]






 70%|███████   | 14/20 [00:02<00:00,  7.28it/s]






 75%|███████▌  | 15/20 [00:02<00:00,  7.50it/s]






 80%|████████  | 16/20 [00:02<00:00,  7.48it/s]






 85%|████████▌ | 17/20 [00:02<00:00,  7.40it/s]



  5%|▌         | 1/20 [00:00<00:02,  8.40it/s]






 10%|█         | 2/20 [00:00<00:02,  8.72it/s]






 15%|█▌        | 3/20 [00:00<00:01,  8.97it/s]






 20%|██        | 4/20 [00:00<00:01,  8.97it/s]






 25%|██▌       | 5/20 [00:00<00:01,  9.03it/s]






 30%|███       | 6/20 [00:00<00:01,  9.21it/s]






 35%|███▌      | 7/20 [00:00<00:01,  9.35it/s]






 40%|████      | 8/20 [00:00<00:01,  9.37it/s]






 45%|████▌     | 9/20 [00:00<00:01,  9.40it/s]






 50%|█████     | 10/20 [00:01<00:01,  9.35it/s]






 55%|█████▌    | 11/20 [00:01<00:00,  9.29it/s]






 60%|██████    | 12/20 [00:01<00:00,  9.19it/s]






 65%|██████▌   | 13/20 [00:01<00:00,  9.29it/s]






 70%|███████   | 14/20 [00:01<00:00,  9.36it/s]






 75%|███████▌  | 15/20 [00:01<00:00,  9.28it/s]






 80%|████████  | 16/20 [00:01<00:00,  9.28it/s]






 85%|████████▌ | 17/20 [00:01<00:00,  9.30it/s]






 90%|█████████ | 18/20 [00:01<00:00,  9.35it/s]






 95%|█████████▌| 19/20 [00:02<00:00, 

 10%|█         | 2/20 [00:00<00:01,  9.03it/s]






 15%|█▌        | 3/20 [00:00<00:01,  9.04it/s]






 20%|██        | 4/20 [00:00<00:01,  9.10it/s]






 25%|██▌       | 5/20 [00:00<00:01,  9.11it/s]






 30%|███       | 6/20 [00:00<00:01,  9.31it/s]






 35%|███▌      | 7/20 [00:00<00:01,  9.44it/s]






 40%|████      | 8/20 [00:00<00:01,  9.50it/s]






 45%|████▌     | 9/20 [00:00<00:01,  9.55it/s]






 50%|█████     | 10/20 [00:01<00:01,  9.38it/s]






 55%|█████▌    | 11/20 [00:01<00:00,  9.33it/s]






 60%|██████    | 12/20 [00:01<00:00,  9.39it/s]






 65%|██████▌   | 13/20 [00:01<00:00,  9.48it/s]






 70%|███████   | 14/20 [00:01<00:00,  9.50it/s]






 75%|███████▌  | 15/20 [00:01<00:00,  9.39it/s]






 80%|████████  | 16/20 [00:01<00:00,  9.45it/s]






 85%|████████▌ | 17/20 [00:01<00:00,  9.37it/s]






 90%|█████████ | 18/20 [00:01<00:00,  9.46it/s]






 95%|█████████▌| 19/20 [00:02<00:00,  9.59it/s]






100%|██████████| 20/20 [00:02<00:00,

 95%|█████████▌| 19/20 [00:02<00:00,  8.76it/s]






100%|██████████| 20/20 [00:02<00:00,  8.63it/s]





 56%|█████▌    | 28/50 [18:21<14:42, 40.14s/it]






  0%|          | 0/20 [00:00<?, ?it/s]






  5%|▌         | 1/20 [00:00<00:02,  7.35it/s]






 10%|█         | 2/20 [00:00<00:02,  7.61it/s]






 15%|█▌        | 3/20 [00:00<00:02,  7.92it/s]






 20%|██        | 4/20 [00:00<00:02,  7.43it/s]






 25%|██▌       | 5/20 [00:00<00:02,  5.98it/s]






 30%|███       | 6/20 [00:00<00:02,  6.39it/s]






 35%|███▌      | 7/20 [00:01<00:01,  6.85it/s]






 40%|████      | 8/20 [00:01<00:01,  6.43it/s]






 45%|████▌     | 9/20 [00:01<00:01,  7.05it/s]






 50%|█████     | 10/20 [00:01<00:01,  7.42it/s]






 55%|█████▌    | 11/20 [00:01<00:01,  7.72it/s]






 60%|██████    | 12/20 [00:01<00:00,  8.03it/s]






 65%|██████▌   | 13/20 [00:01<00:00,  8.23it/s]






 70%|███████   | 14/20 [00:01<00:00,  8.32it/s]






 75%|███████▌  | 15/20 [00:02<00:00,  8.45it/s]

 80%|████████  | 16/20 [00:01<00:00,  9.50it/s]






 85%|████████▌ | 17/20 [00:01<00:00,  8.81it/s]






 90%|█████████ | 18/20 [00:02<00:00,  7.95it/s]






 95%|█████████▌| 19/20 [00:02<00:00,  7.76it/s]






100%|██████████| 20/20 [00:02<00:00,  8.19it/s]





 78%|███████▊  | 39/50 [25:37<07:13, 39.45s/it]






  0%|          | 0/20 [00:00<?, ?it/s]






  5%|▌         | 1/20 [00:00<00:02,  8.44it/s]






 10%|█         | 2/20 [00:00<00:02,  8.79it/s]






 15%|█▌        | 3/20 [00:00<00:01,  9.12it/s]






 25%|██▌       | 5/20 [00:00<00:01,  9.49it/s]






 30%|███       | 6/20 [00:00<00:01,  9.35it/s]






 40%|████      | 8/20 [00:00<00:01,  9.74it/s]






 50%|█████     | 10/20 [00:00<00:00, 10.04it/s]






 55%|█████▌    | 11/20 [00:01<00:00, 10.00it/s]






 65%|██████▌   | 13/20 [00:01<00:00, 10.20it/s]






 75%|███████▌  | 15/20 [00:01<00:00, 10.34it/s]






 85%|████████▌ | 17/20 [00:01<00:00, 10.31it/s]






 95%|█████████▌| 19/20 [00:01<00:00, 10.34it

In [121]:
df8 = dict_to_df(data8)

In [108]:
df8

,name,uid,gender,region,tag,follows,fans,posts
0,Kakakaoo-,1927564525,female,上海,"微博原创视频博主, 时尚达人",379,13210000,2300
1,林小宅-,1965681503,female,广东,"微博vlog博主, 知名时尚博主",411,11030000,2593
2,深夜徐老师,1770418834,female,广东,"微博时尚博主, 微博vlog博主",1407,9380000,2213
3,丁一晨DYC,1810632930,female,北京,"丁小点形象作者, 微博vlog博主, 知名时尚博主, 萌宠视频自媒体, 微博签约自媒体",1398,7860000,6547
4,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
...,...,...,...,...,...,...,...,...
989,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
990,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93
991,不潮徐老师,1957639491,female,浙江,"微博vlog博主, 时尚美妆视频自媒体",420,310000,126
992,jinxin1988,3975365008,male,河南,"知名时尚博主, 微博故事原创作者",10000,310000,10000


In [107]:
df8.to_csv('bloggers8.csv', encoding = 'utf_8_sig')

# MySQL Database Connection

In [16]:
def insertvalue(tablename, dataframe):
    
    cur = connMySQL()
    for i in range(len(dataframe)):
        sqlinsert = 'INSERT INTO ' + tablename + ' (name, uid, gender, region, tag, follows, fans, posts) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'
        cur.execute(sqlinsert, (dataframe['name'][i], int(dataframe['uid'][i]), dataframe['gender'][i], dataframe['region'][i], 
                                dataframe['tag'][i], int(dataframe['follows'][i]), int(dataframe['fans'][i]), int(dataframe['posts'][i])))

## Table 1: 美妆自媒体

In [111]:
df1 = pd.read_csv('bloggers1.csv')

In [112]:
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [113]:
df1

,name,uid,gender,region,tag,follows,fans,posts
0,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,全球时尚,1660209951,female,海外,"微博知名时尚帐号, 时尚美妆视频自媒体",193,14690000,90000
2,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
3,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
4,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
...,...,...,...,...,...,...,...,...
991,玫莉蔻蔡小姐,3978351005,female,海外,"美妆博主, 盛林风行CEO, 国际芳香理疗师, 营养师, 时尚美妆视频自媒体",1328,250000,4276
992,木子kinoco,2354793012,female,海外,"时尚博主, 秒拍达人, 时尚美妆视频自媒体",205,230000,165
993,大野猪精,6188682265,female,天津,"美妆博主, 时尚美妆视频自媒体",567,240000,680
994,Juneaffer阿si,2853310973,female,上海,时尚美妆视频自媒体,108,210000,590


In [11]:
createtableBloggers('bloggers1')

In [17]:
insertvalue('bloggers1', df1)

## Table 2: 时尚自媒体

In [114]:
df2 = pd.read_csv('bloggers2.csv')

In [116]:
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]

In [117]:
df2

,name,uid,gender,region,tag,follows,fans,posts
0,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8920000,909
1,gogoboi,1706372681,male,上海,"时尚达人, 时装专栏作者, 微博签约自媒体",516,9850000,9075
2,全球时尚,1660209951,female,海外,"微博知名时尚帐号, 时尚美妆视频自媒体",193,14690000,90000
3,丁一晨DYC,1810632930,female,北京,"丁小点形象作者, 微博vlog博主, 知名时尚博主, 萌宠视频自媒体, 微博签约自媒体",1398,7860000,6545
4,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
...,...,...,...,...,...,...,...,...
991,英国购物情报,5485714345,female,海外,时尚美妆视频自媒体,362,130000,30000
992,我是你们的老梦,6396740838,female,北京,"美妆博主, 时尚美妆视频自媒体",148,120000,635
993,靓靓everliang,2043341537,female,福建,"美妆博主, 时尚美妆视频自媒体",794,140000,1789
994,SeeyouKeya,5970306441,female,海外,"知名时尚博主, 时尚美妆视频自媒体",263,70000,402


In [48]:
createtableBloggers('bloggers2')

In [49]:
insertvalue('bloggers2', df2)

## Table 3: 美妆达人

In [118]:
df3 = pd.read_csv('bloggers3.csv')

In [119]:
df3 = df3.loc[:, ~df3.columns.str.contains('^Unnamed')]

In [120]:
df3

,name,uid,gender,region,tag,follows,fans,posts
0,美妆达人馆,6042105257,male,北京,微博时尚美妆视频帐号,96,500000,20000
1,美妆达人菌,5712824035,female,北京,知名美妆博主,28,1160000,509
2,美粧达人,5884054391,female,广东,美妆博主,147,90000,248
3,美妆种草达人,5225142517,male,海外,知名美妆博主,42,130000,160000
4,美妆达人小美,6355361363,female,江苏,时尚博主,76,120000,110000
...,...,...,...,...,...,...,...,...
987,Anne时尚美妆大全,5604933347,female,上海,秒拍达人,116,190,38
988,种草达人小订单,6374958449,female,上海,美妆博主,77,351,773
989,David-美肤达人,3759636897,male,海外,美妆博主,2200,380,359
990,時尚達人loklok,5288286325,female,广东,"广州骏鹰贸易有限公司总顾问, 美妆博主",865,281,82


In [113]:
createtableBloggers('bloggers3')

In [114]:
insertvalue('bloggers3', df3)

## Table 4: 时尚达人

In [121]:
df4 = pd.read_csv('bloggers4.csv')

In [122]:
df4 = df4.loc[:, ~df4.columns.str.contains('^Unnamed')]

In [123]:
df4

,name,uid,gender,region,tag,follows,fans,posts
0,时尚达人Simo,1794845531,female,北京,"时尚达人, 北京《瑞丽》杂志社, 时尚, 特约撰稿人, 知名时尚博主",82,7320000,20000
1,时尚达人Danny,6027416124,male,广东,搞笑幽默博主,133,1040000,208
2,时尙达人,5263923753,female,海外,时尚达人,57,10000,3765
3,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8940000,909
4,Pony__朴惠敏,5882998192,female,海外,"时尚达人，MUNMU, INC., 签约美妆艺术家，资深美妆达人",13,10020000,830
...,...,...,...,...,...,...,...,...
989,造型师文静Maggie-,2250918434,female,山东,"时尚达人, 青岛姬夫莱音艺术形象设计联合创始人, 时尚美妆视频自媒体",616,200000,3029
990,piggy史密斯,1842060691,female,海南,时尚达人,117,140000,334
991,小何欧尼_,7024286863,female,海外,时尚达人,46,130000,125
992,猪头脸1,5145779611,female,海外,时尚达人,489,130000,168


In [115]:
createtableBloggers('bloggers4')

In [116]:
insertvalue('bloggers4', df4)

## Table 5: 美妆博主

In [124]:
df5 = pd.read_csv('bloggers5.csv')

In [125]:
df5 = df5.loc[:, ~df5.columns.str.contains('^Unnamed')]

In [126]:
df5

,name,uid,gender,region,tag,follows,fans,posts
0,美粧博主,5881204733,female,其他,知名美妆博主,149,870000,4239
1,美妆博主,5322214912,female,海外,美妆博主,1281,70000,92
2,美妆博主推荐,6019698320,female,江苏,"知名美妆博主, 时尚美妆视频自媒体",153,1700000,20000
3,英國時尚美妆博主LIZ,1743031157,female,海外,"时尚博主, 时尚美妆视频自媒体",457,860000,2074
4,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8940000,909
...,...,...,...,...,...,...,...,...
991,刘施妍,2716629405,female,广东,"OnlyStar签约达人, 时尚达人, 微博vlog博主, 微博故事红人, 时尚美妆视频自媒体",339,620000,596
992,叁月是蒜瓣儿,1908871343,female,北京,"Kimiss闺蜜网资深美容编辑, 知名美妆博主",675,610000,633
993,拆姐搞测评,6522098854,female,北京,知名美妆博主,49,580000,456
994,Elvis增增,3978197240,male,上海,知名美妆博主,73,520000,2159


In [62]:
createtableBloggers('bloggers5')

In [63]:
insertvalue('bloggers5', df5)

## Table 6: 时尚博主

In [131]:
df6 = pd.read_csv('bloggers6.csv')

In [132]:
df6 = df6.loc[:, ~df6.columns.str.contains('^Unnamed')]

In [133]:
df6

,name,uid,gender,region,tag,follows,fans,posts
0,英國時尚美妆博主LIZ,1743031157,female,海外,"时尚博主, 时尚美妆视频自媒体",457,860000,2074
1,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",283,8950000,910
2,Kakakaoo-,1927564525,female,上海,"微博原创视频博主, 时尚达人",379,13210000,2299
3,时尚博主-Rose女士,1822046155,female,北京,", , 北京逸槿国际商贸有限公司Rose, paris首席设计师夏梦莹",379,380000,796
4,买买菌,5098547539,female,北京,"时尚达人, 微博知名博主, 微博原创视频博主, 微博签约自媒体",1324,4070000,10000
...,...,...,...,...,...,...,...,...
972,猫屎弟弟,1559796571,male,北京,"时尚达人, 微博时尚博主, 时装专栏编辑, 知名时尚博主",350,2500000,2146
973,时尚搭配师lucy,2824623903,male,北京,"时尚达人, 微博vlog博主",230,2280000,10000
974,潮美搭师,5867562875,female,山东,知名时尚博主,545,1770000,10000
975,避不避雨,7000738818,male,广东,"微博故事红人, 知名时尚博主",296,1050000,229


In [32]:
createtableBloggers('bloggers6')

In [35]:
insertvalue('bloggers6', df6)

## Table 7: 微博美妆博主

In [134]:
df7 = pd.read_csv('bloggers7.csv')

In [135]:
df7 = df7.loc[:, ~df7.columns.str.contains('^Unnamed')]

In [136]:
df7

,name,uid,gender,region,tag,follows,fans,posts
0,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
1,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5700000,1017
2,虫虫Chonny,2634154091,female,浙江,"时尚达人, 知名美妆博主, 微博签约自媒体",560,4610000,919
3,扇子NO_FAN_NO_FUN,1944603171,female,海外,"微博vlog博主, 快美妆签约红人, 时尚达人, 知名美妆博主",162,2760000,1530
4,管阿姨,1225512093,female,浙江,"微博vlog博主, 杭州柳泉文化创意有限公司, 摄影部总监, 知名时尚博主, 时尚美妆视频自媒体",1182,2510000,7703
...,...,...,...,...,...,...,...,...
846,南京无痕接发苏洋,5081701352,male,江苏,时尚美妆视频自媒体,145,2625,117
847,Vlogger小V,6120686578,female,北京,微博vlog博主,208,2409,89
848,美容护肤师_娜娜,3805931144,female,山西,太原伊诗美商贸有限公司营养护肤老师,986,1776,778
849,陆衫定z7m0,6329772692,female,贵州,美妆博主,517,416,668


In [66]:
createtableBloggers('bloggers7')

In [67]:
insertvalue('bloggers7', df7)

## Table 8: 微博时尚博主

In [137]:
df8 = pd.read_csv('bloggers8.csv')

In [138]:
df8 = df8.loc[:, ~df8.columns.str.contains('^Unnamed')]

In [139]:
df8

,name,uid,gender,region,tag,follows,fans,posts
0,Kakakaoo-,1927564525,female,上海,"微博原创视频博主, 时尚达人",379,13210000,2300
1,林小宅-,1965681503,female,广东,"微博vlog博主, 知名时尚博主",411,11030000,2593
2,深夜徐老师,1770418834,female,广东,"微博时尚博主, 微博vlog博主",1407,9380000,2213
3,丁一晨DYC,1810632930,female,北京,"丁小点形象作者, 微博vlog博主, 知名时尚博主, 萌宠视频自媒体, 微博签约自媒体",1398,7860000,6547
4,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
...,...,...,...,...,...,...,...,...
989,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
990,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93
991,不潮徐老师,1957639491,female,浙江,"微博vlog博主, 时尚美妆视频自媒体",420,310000,126
992,jinxin1988,3975365008,male,河南,"知名时尚博主, 微博故事原创作者",10000,310000,10000


In [18]:
createtableBloggers('bloggers8')

In [19]:
insertvalue('bloggers8', df8)

# Data Preprocessing

## Remove Duplicated Bloggers

### With MySQL

In [71]:
cur = connMySQL()

In [72]:
# combine 8 tables, 5951 bloggers in total
sqlcombine = '''CREATE TABLE bloggerscombine 
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers1 
                UNION ALL
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers2 
                UNION ALL
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers3 
                UNION ALL
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers4 
                UNION ALL
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers5 
                UNION ALL
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers6
                UNION ALL
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers7
                UNION ALL
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggers8
             '''
cur.execute(sqlcombine)

7796

In [73]:
sqlprimarykey = '''ALTER TABLE bloggerscombine ADD COLUMN id INT(10) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY FIRST'''
cur.execute(sqlprimarykey)

0

In [74]:
# remove duplicated rows, 4576 bloggers left
sqlrmdupli = '''CREATE TABLE bloggersfinal 
                SELECT name, uid, gender, region, tag, follows, fans, posts FROM bloggerscombine GROUP BY uid
             '''
sqlprimarykey = '''ALTER TABLE bloggersfinal ADD COLUMN id INT(10) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY FIRST'''

cur.execute(sqlrmdupli)
cur.execute(sqlprimarykey)

4576

0

### Validation With Python

In [148]:
bloggers_final = pd.DataFrame(columns = ['name', 'uid', 'gender', 'region', 'tag', 'follows', 'fans', 'posts'])

for i in range(8):
    data = pd.read_csv('bloggers' + str(i+1) + '.csv')
    data_list = [bloggers_final, data]
    bloggers_final = pd.concat(data_list, axis = 0)

<ipython-input-148-3b7a4c1ee62f>:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  bloggers_final = pd.concat(data_list, axis = 0)


In [149]:
bloggers_final = bloggers_final.drop_duplicates(subset = ['uid'], keep = 'first', inplace = False)

cols = ['name', 'uid', 'gender', 'region', 'tag', 'follows', 'fans', 'posts']
bloggers_final = bloggers_final.loc[:,cols]

bloggers_final = bloggers_final.loc[:, ~bloggers_final.columns.str.contains('^Unnamed')]
bloggers_final

,name,uid,gender,region,tag,follows,fans,posts
0,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,全球时尚,1660209951,female,海外,"微博知名时尚帐号, 时尚美妆视频自媒体",193,14690000,90000
2,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
3,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
4,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
...,...,...,...,...,...,...,...,...
986,AmilyShen,1845376037,female,海外,"知名时尚博主, 微博vlog博主",819,1130000,2326
988,mini咔的咔,3222085612,female,江苏,"去哪儿网, 聪明旅行家, 时尚博主, 微博故事原创作者, 旅行视频自媒体",280,520000,784
989,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
990,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93


In [150]:
len(bloggers_final)

4576

In [92]:
# convert to csv
df = pd.read_excel('bloggersfinal.xlsx', header = None, keep_default_na = False)
df.columns = ['id', 'name', 'uid', 'gender', 'region', 'tag', 'follows', 'fans', 'posts']
df.to_csv('bloggersfinal.csv', encoding = 'utf_8_sig', index = False)

## Remove Bloggers

### Remove MKTers

In [96]:
rmv_kw_dict = ['代购', '日代', '总代', '淘', '省钱', '券', '大牌', '品牌', '囤货', '国货', '优惠', '折扣', '网', '推荐', '安利', '好物', '买', '购物', '剁手', '红包',
               '白菜', '收割机', '蘑菇', '吃瓜',
               '热门', '全球', '环球', '海外', '资讯', '头条', '潮流', '潮装', '潮人', '有范', '范儿', '英伦', '欧美', '日式', '韩系', '日系', '时尚娱乐', '创意',
               '发型', '头发', '美发', '编发', '扎发', '接发', '植发', '染', '烫', '洗', 
               '衣', '服', '时装', '穿', '搭', '街拍', '打扮', '妆容', '潮妆', '造型', '秀', '珠宝', '鞋', '液', '饰品', '唇膏', '香水', '美瞳',
               '美容', '瘦身', '健身', '健康', '养生', '皮肤', '肌肤', '痘', '减肥', '纹身', '美甲', '指甲', '隆鼻', '整形', '医美', '无痕',
               '精选', '技巧', '诀窍', '妙招', '手册', '笔记', '录', '集', '宝典', '指南', '计划', '时尚志', '课堂', '讲堂', '教程', '教学', '教', '知识', '常识', '社', '秘籍', '攻略', '日常', '分享', '收纳', '锦鲤', '关注',
               '图片', '美图', '图片', '壁纸', '封面', '拍', '头像', '视频', '摄像', '摄影', '素材', '资源', '游戏',
               '男士', '女士', '男人', '女人', '男神', '女神', '女王', '校花', '校草', '美妆博主', '时尚博主', '美妆馆', '造型师', '化妆师', '护肤师', '模特', '顾问', '主理人', '总监', '资深', '爱豆',
               '疗伤', '变美', '颜值', '败家', '婚纱', '婚礼', '配方', '娱乐', '明星', '艺术', '关注', '家居', '扒客', '爆料', '昵称', '饭圈', '测评', '评价', '评论', '咨询', '设计', '编辑', '时尚说','搬运',
               '北京', '成都', '泉州', '武汉', '长春', '深圳', '巴黎', '纽约', '全智贤', '业务', '旗舰', '奢侈', '坊', '公司', '品位', '育儿', '司机', '综艺', '影视', '排行', '剧透', '电商', '自己'
               ]

In [97]:
len(df)

4576

In [98]:
delete_index = []
for i in range(len(df)):
    for mkt_act in rmv_kw_dict:
        if mkt_act in list(df['name'])[i]:
            delete_index.append(i)

In [99]:
rmv_list = list(set(delete_index))
len(rmv_list)

784

In [100]:
df = df.drop(df.index[[rmv_list]])
len(df)

3792

In [101]:
df = df.reset_index(drop = True)
df

,id,name,uid,gender,region,tag,follows,fans,posts
0,1,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,3,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
2,4,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
3,5,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
4,6,子望,2395743084,female,其他,"演员, 作家, 时尚美妆视频自媒体",128,3870000,837
...,...,...,...,...,...,...,...,...,...
3787,4572,AmilyShen,1845376037,female,海外,"知名时尚博主, 微博vlog博主",819,1130000,2326
3788,4573,mini咔的咔,3222085612,female,江苏,"去哪儿网, 聪明旅行家, 时尚博主, 微博故事原创作者, 旅行视频自媒体",280,520000,784
3789,4574,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
3790,4575,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93


### Remove Bloggers with 0 Follows, Fans, Posts

In [112]:
delete_index = []
for i in range(len(df)):
    if df['follows'][i] == 0 or df['fans'][i] == 0 or df['posts'][i] == 0:
        delete_index.append(i)

In [113]:
rmv_list = list(set(delete_index))
len(rmv_list)

13

In [114]:
df = df.drop(df.index[[rmv_list]])
len(df)

3779

In [121]:
df = df.reset_index(drop = True)
df

,id,name,uid,gender,region,tag,follows,fans,posts
0,1,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,3,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
2,4,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
3,5,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
4,6,子望,2395743084,female,其他,"演员, 作家, 时尚美妆视频自媒体",128,3870000,837
...,...,...,...,...,...,...,...,...,...
3774,4572,AmilyShen,1845376037,female,海外,"知名时尚博主, 微博vlog博主",819,1130000,2326
3775,4573,mini咔的咔,3222085612,female,江苏,"去哪儿网, 聪明旅行家, 时尚博主, 微博故事原创作者, 旅行视频自媒体",280,520000,784
3776,4574,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
3777,4575,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93


In [125]:
df['id'] = np.arange(1,len(df)+1)
df

<function id(obj, /)>

In [127]:
df.to_csv('bloggers_3779left.csv', encoding = 'utf_8_sig', index = False)

## Remove Unrelated Accounts (Human Labeling)

In [37]:
whole = pd.read_csv("bloggers_3779left.csv")
whole
len(whole) # 3779 bloggers

,id,name,uid,gender,region,tag,follows,fans,posts
0,1,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,2,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
2,3,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
3,4,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
4,5,子望,2395743084,female,其他,"演员, 作家, 时尚美妆视频自媒体",128,3870000,837
...,...,...,...,...,...,...,...,...,...
3774,3775,AmilyShen,1845376037,female,海外,"知名时尚博主, 微博vlog博主",819,1130000,2326
3775,3776,mini咔的咔,3222085612,female,江苏,"去哪儿网, 聪明旅行家, 时尚博主, 微博故事原创作者, 旅行视频自媒体",280,520000,784
3776,3777,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
3777,3778,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93


3779

In [47]:
rmv = pd.read_excel("removed_bloggers.xlsx", encoding = 'utf-8')
rmv = rmv.drop_duplicates()
len(rmv) # 1163 bloggers
rmv_lst = rmv['uid'].tolist()

1163

In [51]:
clean = whole[ ~ whole['uid'].isin(uid for uid in rmv_lst)]
clean['id'] = np.arange(1,len(clean)+1)
clean = tmp.reset_index(drop = True)
clean.to_csv("clean_bloggers.csv", encoding = 'utf_8_sig', index = False)
clean

<ipython-input-51-31b38dc994a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['id'] = np.arange(1,len(clean)+1)


,id,name,uid,gender,region,tag,follows,fans,posts
0,1,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,2,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
2,3,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
3,4,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
4,5,子望,2395743084,female,其他,"演员, 作家, 时尚美妆视频自媒体",128,3870000,837
...,...,...,...,...,...,...,...,...,...
2611,2612,AmilyShen,1845376037,female,海外,"知名时尚博主, 微博vlog博主",819,1130000,2326
2612,2613,mini咔的咔,3222085612,female,江苏,"去哪儿网, 聪明旅行家, 时尚博主, 微博故事原创作者, 旅行视频自媒体",280,520000,784
2613,2614,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
2614,2615,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93


## Append Bloggers

### Duplicate Check

In [156]:
whole_bloggers = pd.read_csv('bloggersfinal.csv')

append_bloggers = pd.read_excel('append_bloggers.xlsx', header = None)
append_bloggers = append_bloggers.drop_duplicates()
append_bloggers.columns = ['name']
append_bloggers = append_bloggers.reset_index(drop = True)
append_bloggers
len(append_bloggers)

,name
0,Kakakaoo-
1,晚晚学姐
2,sodaOoOo
3,一匹英俊的小马
4,Hey是SuSu啊
...,...
308,是蛋妈啊
309,搭搭dada
310,邢晓瑶
311,张沫凡MOMO


313

In [158]:
def check_duplicate(append_list, whole_list):
    new_lst = []
    
    for blogger in append_list:
        blogger = blogger.strip(' ')
        blogger = blogger.strip('#')
        
        if blogger in whole_list:
            continue
            
        new_lst.append(blogger)
        
    return new_lst

In [159]:
new_lst = check_duplicate(append_bloggers['name'].tolist(), whole_bloggers['name'].tolist())
len(new_lst)

175

In [160]:
# 175 bloggers will be appended to the list
new_bloggers = pd.DataFrame(data = new_lst)
new_bloggers.columns = ['name']
new_bloggers.to_csv('new_bloggers.csv', encoding = 'utf_8_sig')
new_bloggers

,name
0,sodaOoOo
1,一匹英俊的小马
2,就是曼仔
3,佩里邱
4,走向世界的彭美丽
...,...
170,是蛋妈啊
171,搭搭dada
172,邢晓瑶
173,张沫凡MOMO


### Scrapy Appended Bloggers Info

In [161]:
newbloggerls = new_bloggers['name'].tolist()
len(newbloggerls)

175

In [162]:
def append_bloggers(username, password, bloggerlist):
    
    driver = webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')
    driver.get('http://weibo.com/login.php')
    
    driver.find_element_by_name('username').send_keys(username)
    driver.find_element_by_name('password').send_keys(password)
    time.sleep(5) # time to input validation code
    
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/div/div/div[3]/div[6]').click()
    time.sleep(5)
    
    data = {}
    
    for i in tqdm(range(len(bloggerlist))):
        
        driver.get('https://s.weibo.com/')
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/a[2]').click()
        time.sleep(5)

        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').clear()
        topic = bloggerlist[i]
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').send_keys(topic)
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/button').click()
        time.sleep(5)
        
        driver.find_element_by_link_text('找人').click()
        time.sleep(5)
        
        try:
            table = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/div[1]/div[2]')
            
            if table.find_element_by_xpath('div/a[1]').text != bloggerlist[i]:
                continue
                
            name = table.find_element_by_xpath('div/a[1]').text   
            gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
            gender = re.findall('icon-sex-(.*)', gender)[0]
            region = table.find_element_by_xpath('p[1]').text.split(' ')[0]
            
            if table.find_element_by_xpath('p[3]').text[:2] == '关注':
                uid = table.find_element_by_xpath('p[3]/span[1]/a').get_attribute('href')
                uid = re.findall('[0-9]+', uid)[0]
                tag = table.find_element_by_xpath('p[2]').text.split(' ')
                follows = table.find_element_by_xpath('p[3]/span/a').text
                fans = table.find_element_by_xpath('p[3]/span[2]/a').text
                posts = table.find_element_by_xpath('p[3]/span[3]/a').text
                
                
            
            else:
                uid = table.find_element_by_xpath('p[2]/span[1]/a').get_attribute('href')
                uid = re.findall('[0-9]+', uid)[0]
                tag = []
                follows = table.find_element_by_xpath('p[2]/span/a').text
                fans = table.find_element_by_xpath('p[2]/span[2]/a').text
                posts = table.find_element_by_xpath('p[2]/span[3]/a').text
                
            data[len(data)] = {'name':name, 'uid':uid, 'gender':gender, 'region':region,
                               'tag':tag, 'follows':follows, 'fans':fans, 'posts':posts}
       
        except:
            pass

    return data

In [164]:
data9 = append_bloggers('lanjiao46505@163.com', 'asd123123a', newbloggerls)


100%|██████████| 175/175 [1:17:38<00:00, 26.62s/it]


In [165]:
df9 = dict_to_df(data9)
df9

,name,uid,gender,region,tag,follows,fans,posts
0,sodaOoOo,5682369421,female,北京,,219,110000,171
1,一匹英俊的小马,5228895290,female,海外,,855,670000,2604
2,走向世界的彭美丽,2766134004,female,北京,微博vlog博主,482,2080000,395
3,你好_竹子,1763998647,female,海外,"知名摄影博主, 微博vlog博主, 视频自媒体",712,3520000,2335
4,不是椰奶,5947101696,female,其他,,565,510000,2301
...,...,...,...,...,...,...,...,...
164,是蛋妈啊,1876516967,female,海外,"秒拍达人, 微博vlog博主",1714,1750000,30000
165,搭搭dada,1888244903,female,浙江,时尚红人，URDADA品牌主理人,91,2430000,1609
166,邢晓瑶,1557111167,female,上海,模特邢瑶，合作媒体《EASY》《YOHO潮流志》等,556,5720000,1773
167,张沫凡MOMO,1780449083,female,北京,"美沫艾莫尔创始人、CEO, 微博故事红人",735,12480000,4625


In [169]:
missing_lst = []
for i in newbloggerls:
    if i not in df9['name'].tolist():
        missing_lst.append(i)
missing_lst

['就是曼仔', '佩里邱', '粉乳乳', '小米苏酥', '可可壹-', '肉饼子登蛋']

In [174]:
data10 = append_bloggers('lanjiao46505@163.com', 'asd123123a', missing_lst)


100%|██████████| 6/6 [01:46<00:00, 17.83s/it]


In [175]:
df10 = dict_to_df(data10)
df10

,name,uid,gender,region,tag,follows,fans,posts
0,就是曼仔,1392659117,female,海外,,118,260000,4903
1,佩里邱,5911172477,female,海外,,96,790000,89
2,小米苏酥,87768787,female,海外,,594,510000,10000


In [177]:
df11 = df9.append(df10)
df11 = df11.reset_index(drop = True)
df11

,name,uid,gender,region,tag,follows,fans,posts
0,sodaOoOo,5682369421,female,北京,,219,110000,171
1,一匹英俊的小马,5228895290,female,海外,,855,670000,2604
2,走向世界的彭美丽,2766134004,female,北京,微博vlog博主,482,2080000,395
3,你好_竹子,1763998647,female,海外,"知名摄影博主, 微博vlog博主, 视频自媒体",712,3520000,2335
4,不是椰奶,5947101696,female,其他,,565,510000,2301
...,...,...,...,...,...,...,...,...
167,张沫凡MOMO,1780449083,female,北京,"美沫艾莫尔创始人、CEO, 微博故事红人",735,12480000,4625
168,郑能量姑娘Goya,1151593834,female,北京,"知名旅游博主, 旅行玩家, 去哪儿网聪明旅行家, 微博vlog博主, 头条文章作者, 微博签...",329,1990000,1460
169,就是曼仔,1392659117,female,海外,,118,260000,4903
170,佩里邱,5911172477,female,海外,,96,790000,89


In [103]:
col_name = df11.columns.tolist()
col_name.insert(col_name.index('name'), 'id')
df11 = df11.reindex(columns = col_name)
df11['id'] = np.arange(1, len(df11)+1)
df11

,id,name,uid,gender,region,tag,follows,fans,posts
0,1,sodaOoOo,5682369421,female,北京,,219,110000,171
1,2,一匹英俊的小马,5228895290,female,海外,,855,670000,2604
2,3,走向世界的彭美丽,2766134004,female,北京,微博vlog博主,482,2080000,395
3,4,你好_竹子,1763998647,female,海外,"知名摄影博主, 微博vlog博主, 视频自媒体",712,3520000,2335
4,5,不是椰奶,5947101696,female,其他,,565,510000,2301
...,...,...,...,...,...,...,...,...,...
167,168,张沫凡MOMO,1780449083,female,北京,"美沫艾莫尔创始人、CEO, 微博故事红人",735,12480000,4625
168,169,郑能量姑娘Goya,1151593834,female,北京,"知名旅游博主, 旅行玩家, 去哪儿网聪明旅行家, 微博vlog博主, 头条文章作者, 微博签...",329,1990000,1460
169,170,就是曼仔,1392659117,female,海外,,118,260000,4903
170,171,佩里邱,5911172477,female,海外,,96,790000,89


In [106]:
df11.to_csv('172bloggers_append.csv', index = False, encoding = 'utf_8_sig')

# Finalize Bloggers

In [109]:
clean_df = pd.read_csv("clean_bloggers.csv")
clean_df = clean_df.loc[:, ~clean_df.columns.str.contains('^Unnamed')]
clean_df

append_df = pd.read_csv("172bloggers_append.csv", keep_default_na = False)
append_df = append_df.loc[:, ~append_df.columns.str.contains('^Unnamed')]
append_df

,id,name,uid,gender,region,tag,follows,fans,posts
0,1,李佳琦Austin,1968758563,male,上海,"美ONE签约达人, 知名美妆博主, 时尚美妆视频自媒体",282,8910000,907
1,2,牛尔,1740006601,male,台湾,"时尚达人, 台湾知名美容专家, 知名美妆博主, 头条文章作者, 微博签约自媒体",845,27340000,20000
2,3,张凯毅KEVIN,2255058517,female,北京,"时尚达人, 时尚美妆视频自媒体",128,4900000,451
3,4,仇仇-qiuqiu,1573953470,female,上海,"时尚达人, 微博vlog博主, 知名美妆博主, 时尚美妆视频自媒体, 微博签约自媒体",541,5690000,1014
4,5,子望,2395743084,female,其他,"演员, 作家, 时尚美妆视频自媒体",128,3870000,837
...,...,...,...,...,...,...,...,...,...
2611,2612,AmilyShen,1845376037,female,海外,"知名时尚博主, 微博vlog博主",819,1130000,2326
2612,2613,mini咔的咔,3222085612,female,江苏,"去哪儿网, 聪明旅行家, 时尚博主, 微博故事原创作者, 旅行视频自媒体",280,520000,784
2613,2614,斯科特BOY,5887161590,male,海外,"微博搞笑视频博主, 时尚博主",269,520000,34
2614,2615,洋子姐yangzi,5271828555,female,安徽,"微博牧场计划合作达人, 时尚博主",208,540000,93


,id,name,uid,gender,region,tag,follows,fans,posts
0,1,sodaOoOo,5682369421,female,北京,,219,110000,171
1,2,一匹英俊的小马,5228895290,female,海外,,855,670000,2604
2,3,走向世界的彭美丽,2766134004,female,北京,微博vlog博主,482,2080000,395
3,4,你好_竹子,1763998647,female,海外,"知名摄影博主, 微博vlog博主, 视频自媒体",712,3520000,2335
4,5,不是椰奶,5947101696,female,其他,,565,510000,2301
...,...,...,...,...,...,...,...,...,...
167,168,张沫凡MOMO,1780449083,female,北京,"美沫艾莫尔创始人、CEO, 微博故事红人",735,12480000,4625
168,169,郑能量姑娘Goya,1151593834,female,北京,"知名旅游博主, 旅行玩家, 去哪儿网聪明旅行家, 微博vlog博主, 头条文章作者, 微博签...",329,1990000,1460
169,170,就是曼仔,1392659117,female,海外,,118,260000,4903
170,171,佩里邱,5911172477,female,海外,,96,790000,89


In [121]:
finalized_bloggers = clean_df.append(append_df).drop_duplicates()
finalized_bloggers = finalized_bloggers.sort_index(axis = 0, ascending = True, by = ['uid'])
finalized_bloggers['id'] = np.arange(1, len(finalized_bloggers)+1)
finalized_bloggers = finalized_bloggers.reset_index(drop = True)
finalized_bloggers.to_csv('finalized_2788bloggers.csv', index = False, encoding = 'utf_8_sig')
finalized_bloggers

<ipython-input-121-07a458bd0ade>:2: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  finalized_bloggers = finalized_bloggers.sort_index(axis = 0, ascending = True, by = ['uid'])


,id,name,uid,gender,region,tag,follows,fans,posts
0,1,小米苏酥,87768787,female,海外,,594,510000,10000
1,2,萌大雨YUYU,1011269330,female,其他,"时尚达人, 知名美妆博主, 时尚美妆视频自媒体",57,3870000,1343
2,3,missfaye,1032937413,female,北京,"时尚达人, 知名时尚博主, 微博尤物志合作达人",1467,800000,20000
3,4,kittywenny,1036701494,female,上海,"资深时尚美容达人, 微博尤物志合作达人, 微博vlog博主, 微博签约自媒体",986,1390000,7502
4,5,羽晗winnie,1039619890,female,北京,"知名旅行玩家美食达人, 微博vlog博主, 知名时尚博主, 头条文章作者, 微博签约自媒体",321,3940000,1195
...,...,...,...,...,...,...,...,...,...
2783,2784,胡宾果Bingo,7262819693,male,浙江,时尚美妆视频自媒体,3,890000,241
2784,2785,DrRay的美妆情报局,7276052993,female,上海,美妆博主,51,250000,61
2785,2786,宝爷Beauty,7279837894,female,广东,"美妆博主, 时尚美妆视频自媒体",18,50000,31
2786,2787,沪上兄弟,7294070020,male,上海,"美妆博主, 微博故事原创作者",63,100000,57
